## Segmenting and Clustering Neighborhoods in Toronto - Project

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [2]:
#Installing html5lib for html parsing
pip install html5lib

Note: you may need to restart the kernel to use updated packages.


In [3]:
#Installing lxml library for processing xml and html
pip install lxml


     |████████████████████████████████| 5.5MB 5.9MB/s eta 0:00:01     |████████▎                       | 1.4MB 5.9MB/s eta 0:00:01     |████████████████████████▉       | 4.3MB 5.9MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


## 1. Download and Explore Dataset

The project uses the Toronto neighborhood data extracted from: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M,

In [1]:
import pandas as pd

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
dfs = pd.read_html(url)
dfs[0]

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


* The dataframe consists of three columns: PostalCode, Borough, and Neighborhood
* Only processing the cells that have an assigned borough and ignoring cells with a borough that is "Not assigned".
* More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [2]:
toronto_data = dfs[0][dfs[0]['Borough']!='Not assigned']
toronto_data =  toronto_data.reset_index(drop=True)


In [19]:
toronto_data.shape

(103, 3)

In [12]:
toronto_data.describe()

,Postal Code,Borough,Neighborhood
count,103,103,103
unique,103,10,99
top,M4G,North York,Downsview
freq,1,24,4


As you can see the shape of the final processes data has 103 unique postal codes, 10 unique Boroughs and 99 unique Neigbhorhoods**

*Note: Neighborhood as explained above has instances with shared postal code as in the case of M5A, this is the current state and will be enhanced in the code below*